# Coronavirus Data Analysis

In [1]:
import pandas as pd
import numpy as np

In [2]:
spark.sparkContext.applicationId

'application_1636170550178_0040'

In [3]:
pd.options.display.max_columns = 35

### Create Variables for Queries

In [4]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [5]:
stdVar = spark.sql(f"""
  select  round(std(h.length_of_stay),2) as std_length_of_stay
       
  from {userName}.{algorithmName}_Hospital h
  
  left join {userName}.{algorithmName}_Vaccine v
    on h.personid = v.personId
      and h.tenant = v.tenant

   where    
      datediff(h.serviceDate, h.dischargeDate) < 0
      and h.length_of_stay > 0
               
      and  ( datediff(h.serviceDate, v.vaccine_date) > 0
             and v.vaccine_date is not null)        
      
    
""")
stdValue_vac = stdVar.first()['std_length_of_stay']
print(stdValue_vac)

5.62


In [6]:
spark.sql(f"""
    select  discharge_to
          , vaccine_name
          , count(distinct personId) as count_of_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
        from (  
          select
              h.personId
              , length_of_stay
              , case 
                when taken_Moderna_vaccine = 1 then "Moderna"
                when taken_Pfizer_vaccine = 1 then "Pfizer"
                when taken_Janssen_vaccine = 1 then "Janssen"
              end as vaccine_name 
              ,case
                    when discharged_to_skilled_nursing_facility = 1 then "Skilled Nursing Facility"
                    when discharged_to_hospice = 1 then 'Hospice Care'
                    when died_in_the_hospital = 1 then 'Mortuary'
                    when discharged_to_rehab_facility = 1 then 'Rehab Facility'
                    when discharged_to_long_term_care_facility = 1 then 'Long Term Care Facility'
                else 'Home'
                end as discharge_to
              
        from {userName}.{algorithmName}_Hospital h
          
        join {userName}.{algorithmName}_Vaccine_Detail v
            on h.personid = v.personId
              and h.tenant = v.tenant
              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
              and     v.vaccine_date is not null) 
              and ( taken_Moderna_vaccine = 1   
                    or taken_Pfizer_vaccine = 1  
                    or taken_Janssen_vaccine = 1 )

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}
              
              and deceased = 1

          ) subquery   
          
        group by 1, 2
        order by 1, 2
     
""").toPandas()

,discharge_to,vaccine_name,count_of_vaccinated_patients,mean_length_of_stay
0,Home,Moderna,1,3.00
1,Home,Pfizer,1,5.00
2,Hospice Care,Moderna,2,5.50
3,Mortuary,Moderna,5,4.57
4,Mortuary,Pfizer,3,4.00


In [7]:
spark.sql(f"""
    select  vaccine_name
          , count(distinct personId) as count_of_vaccinated_patients
          , round(mean(length_of_stay),2) as mean_length_of_stay
        from (  
          select
              h.personId
              , length_of_stay
              , case 
                when taken_Moderna_vaccine = 1 then "Moderna"
                when taken_Pfizer_vaccine = 1 then "Pfizer"
                when taken_Janssen_vaccine = 1 then "Janssen"
              end as vaccine_name 
           
              
        from {userName}.{algorithmName}_Hospital h
          
        join {userName}.{algorithmName}_Vaccine_Detail v
            on h.personid = v.personId
              and h.tenant = v.tenant
              and  ( datediff(h.serviceDate, v.vaccine_date) > 0
              and     v.vaccine_date is not null) 
              and ( taken_Moderna_vaccine = 1   
                    or taken_Pfizer_vaccine = 1  
                    or taken_Janssen_vaccine = 1 )

           where    
              datediff(h.serviceDate, h.dischargeDate) < 0
              and h.length_of_stay > 0
              and h.length_of_stay < 2*{stdValue_vac}
              
              and deceased = 1

          ) subquery   
          
        group by 1 
        order by 1 
     
""").toPandas()

,vaccine_name,count_of_vaccinated_patients,mean_length_of_stay
0,Moderna,8,4.60
1,Pfizer,3,4.25
